In [1]:
addpath("ihMT_steadystate/src")
addpath("ihMT_steadystate/lib")
addpath('ihMT_steadystate/bin')
addpath('ihMT_steadystate/experiments')


In [2]:
%% 19-4-2019: This script makes Figure 3 from the paper
% Shaihan Malik, King's College London, 2019


theta = deg2rad(35);
dur = 2e-3;
TR = 5e-3;
b1rms = 2;
delta = 7e3;
nband = 2;
dt=6e-6;
gauss_alpha = 3;

pulses={};
pulses{1} = gen_MB_pulse(theta,dur,TR,b1rms,delta,'2+','alpha',gauss_alpha,'dt',dt);
pulses{2} = gen_MB_pulse(theta,dur,TR,b1rms,delta,'2-','alpha',gauss_alpha,'dt',dt);
pulses{3} = gen_MB_pulse(theta,dur,TR,b1rms,delta,'3','alpha',gauss_alpha,'dt',dt);



%% Examine RF power in spectral bands
ff = linspace(-10e3,10e3,1000)';
df=ff(2)-ff(1);
nt = length(pulses{1});%<- all same length
tt = dt*(1:nt);
F = exp(-1i*2*pi*ff*tt)*(dt*1e3)/sqrt(numel(ff)); %<- time in ms

b1sqrd_tau = {}; % integral of B1^2 dt, units uT^2 ms
b1sqrd = {}; % average of above, over pulse duration

% frequency bands to look in
band_ix = {};
bw=2e3;
band_ix{1} = find((ff>-(delta+bw/2))&(ff<-(delta-bw/2)));
band_ix{2} = find((ff>-bw/2)&(ff<bw/2));
band_ix{3} = find((ff>(delta-bw/2))&(ff<(delta+bw/2)));

for ii=1:3
    
    % power spectrum in F
    pwr_spec = abs(F*pulses{ii}).^2;
    
    % look at power in each band - this is scaled to have units
    % equivalent to B1^2 in uT^2
    b1sqrd_tau{ii} = zeros([1 3]);
    for kk=1:3
        b1sqrd_tau{ii}(kk) = sum(pwr_spec(band_ix{kk}))*df;
    end
    tau = 1e3*dt*nt; %<-- this is constant
    b1sqrd{ii} = b1sqrd_tau{ii}/tau;
    
end
        



Flip = 35.0, B1 max = 7.920 uT, B1 rms = 2.000 uT
Flip = 35.0, B1 max = 7.920 uT, B1 rms = 2.000 uT
Flip = 35.0, B1 max = 10.029 uT, B1 rms = 2.000 uT


In [3]:
%get pulses --from MATLAB
%get F --from MATLAB
%get ff --from MATLAB
%get tt --from MATLAB

In [9]:
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import plotly.graph_objects as go

pulses = np.array(pulses)
F = np.array(F)
ff = np.array(ff)
tt = np.array(tt)

In [10]:
fig = make_subplots(rows=3, cols=2, subplot_titles=(f'2\N{SUPERSCRIPT PLUS SIGN} bands, Power Spectrum', 'time domain',
                                                    f'2\N{SUPERSCRIPT MINUS} bands, Power Spectrum', 'time domain',
                                                   f'3 bands, Power Spectrum', 'time domain'), vertical_spacing=0.15)
# plot figures
for i in range(1,4):
    pwr = np.power(abs(F.dot(pulses[i-1])),2)
    
    # plot left column
    fig.append_trace(go.Scatter(x=ff*float(1e-3), y=pwr, line=dict(color="black"), showlegend=False), row=i, col=1)

    idx = (ff>float(-1e3)) & (ff<float(1e3))
    fig.append_trace(go.Scatter(x=ff[idx]*float(1e-3), y=pwr[idx], fill='tozeroy', line=dict(color='green'), showlegend=False), row=i, col=1)
    
    
    idx = (ff>float(6e3)) & (ff<float(8e3))    
    fig.append_trace(go.Scatter(x=ff[idx]*float(1e-3), y=pwr[idx], fill='tozeroy', line=dict(color='red'), showlegend=False), row=i, col=1)
    
    idx = (ff>float(-8e3)) & (ff<float(-6e3))    
    fig.append_trace(go.Scatter(x=ff[idx]*float(1e-3), y=pwr[idx], fill='tozeroy', line=dict(color='red'), showlegend=False), row=i, col=1)
    
    # plot right column
    fig.append_trace(go.Scatter(x=tt*float(1e3), y=pulses[i-1].real, line=dict(color="royalblue"), name='real', ), row=i, col=2)
    fig.append_trace(go.Scatter(x=tt*float(1e3), y=pulses[i-1].imag, line=dict(color="orange"), name='imaginary',), row=i, col=2)


# figure styling
# ticks, names, colors, legends...
fig['layout']['xaxis']['title'] = f'\N{GREEK CAPITAL LETTER DELTA}/2\N{GREEK SMALL LETTER PI} Hz'
fig['layout']['yaxis']['title'] = 'power (au)'

fig['layout']['xaxis3']['title'] = f'\N{GREEK CAPITAL LETTER DELTA}/2\N{GREEK SMALL LETTER PI} Hz'
fig['layout']['yaxis3']['title'] = 'power (au)'

fig['layout']['xaxis5']['title'] = f'\N{GREEK CAPITAL LETTER DELTA}/2\N{GREEK SMALL LETTER PI} Hz'
fig['layout']['yaxis5']['title'] = 'power (au)'

fig['layout']['xaxis2']['title'] = f'\N{GREEK SMALL LETTER TAU}, ms'
fig['layout']['yaxis2']['title'] = f'B\N{SUBSCRIPT ONE}, \N{GREEK SMALL LETTER MU}T'
fig['layout']['xaxis2']['tick0'] = 0
fig['layout']['xaxis2']['dtick'] = 0.5

fig['layout']['xaxis4']['title'] = f'\N{GREEK SMALL LETTER TAU}, ms'
fig['layout']['yaxis4']['title'] = f'B\N{SUBSCRIPT ONE}, \N{GREEK SMALL LETTER MU}T'
fig['layout']['xaxis4']['tick0'] = 0
fig['layout']['xaxis4']['dtick'] = 0.5

fig['layout']['xaxis6']['title'] = f'\N{GREEK SMALL LETTER TAU}, ms'
fig['layout']['yaxis6']['title'] = f'B\N{SUBSCRIPT ONE}, \N{GREEK SMALL LETTER MU}T'
fig['layout']['xaxis6']['tick0'] = 0
fig['layout']['xaxis6']['dtick'] = 0.5


# text on green and red parts
fig.append_trace(go.Scatter(
    x=[-0.2, 6.7, -7.3],
    y=[0.002, 0.0025, 0.0025],
    mode="text",
    text=[1, f'\N{GREEK CAPITAL LETTER BETA}',''], showlegend=False), row=1, col=1)

fig.append_trace(go.Scatter(
    x=[-0.2, 6.7, -7.3],
    y=[0.002, 0.0025, 0.0025],
    mode="text",
    text=[1,'', f'\N{GREEK CAPITAL LETTER BETA}'], showlegend=False), row=2, col=1)

fig.append_trace(go.Scatter(
    x=[-0.2, 6.7, -7.3],
    y=[0.002, 0.0025, 0.0025],
    mode="text",
    text=[1, f'\N{GREEK CAPITAL LETTER BETA}/2',f'\N{GREEK CAPITAL LETTER BETA}/2'], showlegend=False), row=3, col=1)

fig.update_layout(height=800, width=1200, title_text="Side By Side Subplots")
fig.show()



In [ ]:
%% 19-4-2019: This script performs simulations of bSSFP and SPGR for white matter
% Shaihan Malik, King's College London, 2019
% EDITS: 16-6-2019

%% Generates figures 4 and 8 from the paper

tissuepars = init_tissue('ic');


%%% Set up sequence

%%% generate some pulses
flips = deg2rad([1:2:30 35:5:80]);
nf = length(flips);
tau = 2e-3;
TR = 5e-3;
b1_rms = 5;
delta = 7e3;
Delta_Hz = [-delta 0 delta];
dt = 10e-6;
gauss_alpha = 3;

pulses = {};
b1sqrd = {};
b1pulse= {};
for ii=1:nf
    % 3 bands
    [pulses{ii,1},b1sqrd{ii,1},b1pulse{ii,1},TBP] = gen_MB_pulse(flips(ii),tau,TR,b1_rms,delta,'3','alpha',gauss_alpha,'dt',dt); 
    % 2 bands
    [pulses{ii,2},b1sqrd{ii,2},b1pulse{ii,2},TBP] = gen_MB_pulse(flips(ii),tau,TR,b1_rms,delta,'2+','alpha',gauss_alpha,'dt',dt);
end


%% Simulate for both  sequences 

f = [0 1];

Mssfp = zeros(nf,3,2); % FA, #bands, f values
Mspgr = zeros(nf,3,2);

for IX = 1:2
    
    tmp = tissuepars;
    tmp.semi.f = f(IX);
    
    %%% Bieri-Scheffler correction
    tmp = Bieri_Scheffler_finite_correction(TBP,tau,TR,tmp);
    
    %%% ====== bSSFP ==========
    dphi=0;
    for ii=1:nf
        %%% Symmetric (3-band) case
        Mssfp(ii,3,IX) = ssSSFP_ihMT(flips(ii),b1sqrd{ii,1},Delta_Hz,TR,tau,dphi,tmp);
        %%% Asymmetric (2-band) case
        Mssfp(ii,2,IX) = ssSSFP_ihMT(flips(ii),b1sqrd{ii,2},Delta_Hz,TR,tau,dphi,tmp);
        %%% Single band case
        Mssfp(ii,1,IX) =ssSSFP_ihMT(flips(ii),b1sqrd{ii,1}.*[0 1 0],Delta_Hz,TR,tau,dphi,tmp);
    end
    
    %%% ====== SPGR ==========
    for ii=1:nf
        %%% Symmetric (3-band) case
        Mspgr(ii,3,IX) = ssSPGR_ihMT(flips(ii),b1sqrd{ii,1},Delta_Hz,TR,tau,tmp);
        %%% Asymmetric (2-band) case
        Mspgr(ii,2,IX) = ssSPGR_ihMT(flips(ii),b1sqrd{ii,2},Delta_Hz,TR,tau,tmp);
        %%% Single band case
        Mspgr(ii,1,IX) =ssSPGR_ihMT(flips(ii),b1sqrd{ii,1}.*[0 1 0],Delta_Hz,TR,tau,tmp);
    end
    
end

%% Figure 4 from paper 

figfp(1)
nr=2;nc=2;

for jj=1:2
    
    subplot(nr,nc,1+(jj-1)*nc)
    plot(rad2deg(flips),abs(Mssfp(:,:,jj)),'linewidth',1.5);
    grid on
    xlabel('Flip angle, deg')
    ylabel('Signal / M_0')
    pl = get(gca,'Children');
    pl(3).Color = [0 0 0];
    pl(1).LineStyle = '--';
    pl(1).Color = [0 0 1];
    pl(2).Color = [0 1 0];
    
    hold on

    
    subplot(nr,nc,2+(jj-1)*nc)
    plot(rad2deg(flips),abs(Mspgr(:,:,jj)),'linewidth',1.5)
    grid on
    xlabel('Flip angle, deg')
    ylabel('Signal / M_0')
    
    pl = get(gca,'Children');
    pl(3).Color = [0 0 0];
    pl(1).LineStyle = '--';
    pl(1).Color = [0 0 1];
    pl(2).Color = [0 1 0];
    hold on
    
end

% add legend;
ll = legend('S_{1B}','S_{2B}','S_{3B}');
ll.AutoUpdate = 'off';

%%% Now fill in some areas
gg = get(gcf,'children');

%%% For first plot fill in classic MT difference
axes(gg(5))
inBetween = [abs(Mssfp(:,1,1)); flip(abs(Mssfp(:,2,1)),1)];
x2 = [rad2deg(flips) flip(rad2deg(flips),2)];
fh=fill(x2, inBetween, 'b');
fh.EdgeColor = 'none';
fh.FaceAlpha=0.2;

axes(gg(4))
inBetween = [abs(Mspgr(:,1,1)); flip(abs(Mspgr(:,2,1)),1)];
x2 = [rad2deg(flips) flip(rad2deg(flips),2)];
fh=fill(x2, inBetween, 'b');
fh.EdgeColor = 'none';
fh.FaceAlpha=0.2;

axes(gg(3))
inBetween = [abs(Mssfp(:,2,2)); flip(abs(Mssfp(:,3,2)),1)];
x2 = [rad2deg(flips) flip(rad2deg(flips),2)];
fh=fill(x2, inBetween, 'g');
fh.EdgeColor = 'none';
fh.FaceAlpha=0.2;

axes(gg(2))
inBetween = [abs(Mspgr(:,2,2)); flip(abs(Mspgr(:,3,2)),1)];
x2 = [rad2deg(flips) flip(rad2deg(flips),2)];
fh=fill(x2, inBetween, 'g');
fh.EdgeColor = 'none';
fh.FaceAlpha=0.2;

%%% text labels
axes(gg(5))
title('bSSFP','fontsize',16,'fontweight','bold')
text(-25,0.06,'\delta = 0','fontsize',18,'fontweight','bold','rotation',90,'fontangle','italic')

axes(gg(4))
title('SPGR','fontsize',16,'fontweight','bold')

axes(gg(3))
text(-25,0.06,'\delta = 1','fontsize',18,'fontweight','bold','rotation',90,'fontangle','italic')

%%% annotation
a1 = annotation('textarrow',[0.2786 0.2214],[0.6878 0.7715],'String','\DeltaMT','fontsize',13);
a2 = annotation('textarrow',[0.3181 0.2609],[0.2121 0.2958],'String','\DeltaihMT','fontsize',13);
a1.HeadStyle = 'cback1';
a2.HeadStyle = 'cback1';
a1.HeadWidth = 6;
a2.HeadWidth = 6;
%
for ii=2:5
    gg(ii).Color = [1 1 1]*0.95;
end
setpospap([100 100 777 590])
set(gcf, 'InvertHardcopy', 'off')
print -dpng -r300 figs/simplefigure.png

%% Add supporting information figure on ihMTR and B1rms
figfp(2)

subplot(3,1,1)
plot(rad2deg(flips),100*(abs(Mspgr(:,2,2))-abs(Mspgr(:,3,2))),'linewidth',1.5)
grid on
hold on
plot(rad2deg(flips),100*(abs(Mssfp(:,2,2))-abs(Mssfp(:,3,2))),'linewidth',1.5)
ylim([0 1.4])
xlabel('Flip angle, deg')
ylabel('\DeltaihMT (percent of M_0)')
title('\DeltaihMT','fontsize',14,'fontweight','bold')
ll=legend('SPGR','bSSFP');ll.FontSize=11;

subplot(3,1,2)
plot(rad2deg(flips),100*(abs(Mspgr(:,2,2))-abs(Mspgr(:,3,2)))./abs(Mspgr(:,1,2)),'linewidth',1.5)
grid on
hold on
plot(rad2deg(flips),100*(abs(Mssfp(:,2,2))-abs(Mssfp(:,3,2)))./abs(Mssfp(:,1,2)),'linewidth',1.5)

xlabel('Flip angle, deg')
ylabel('ihMTR  (% of S_{1B})')
title('ihMTR','fontsize',14,'fontweight','bold')
ll=legend('SPGR','bSSFP');ll.FontSize=11;



subplot(3,1,3)
b1offres=[];for ii=1:nf,b1offres(ii)=sqrt(b1sqrd{ii,2}(3)*tau/TR);end
plot(rad2deg(flips),b1offres,'linewidth',1.5)
grid on
xlabel('Flip angle, deg')
ylabel('$\sqrt<B_1^2(\Delta\neq 0)>,\mu T$','interpreter','Latex')
title('Off-resonance B_1^{rms}','fontsize',14,'fontweight','bold')
ylim([0 6]);

setpospap([00 00 400 700])
print -dpng -r300 figs/simplefigure_supportinginfo.png

%% Comparison for different frequency offsets and RMS B1, SSFP only

%%% Set up sequence

%%% generate some pulses
flipangle = deg2rad(30);

n=32;
b1rms = linspace(2,8,n);
deltaf= linspace(1000,16e3,n);

tau = 2e-3;
TR = 5e-3;
dt = 10e-6;

pulses = {};
b1sqrd = {};
b1pulse= {};
b1max=[];
for ii=1:n
    for jj=1:n
        % 3 bands
        [pulses{ii,jj,1},b1sqrd{ii,jj,1},b1pulse{ii,jj,1},TBP] = gen_MB_pulse(flipangle,tau,TR,b1rms(ii),deltaf(jj),'3','alpha',gauss_alpha,'dt',dt);
        % 2 bands
        [pulses{ii,jj,2},b1sqrd{ii,jj,2},b1pulse{ii,jj,2},TBP] = gen_MB_pulse(flipangle,tau,TR,b1rms(ii),deltaf(jj),'2+','alpha',gauss_alpha,'dt',dt);
    end
    b1max(ii,1)=max(abs(pulses{ii,n,1}));
    b1max(ii,2)=max(abs(pulses{ii,n,2})); %<-- should not depend on frequency offset
end

%%% Bieri-Scheffler correction
tmp = Bieri_Scheffler_finite_correction(TBP,tau,TR,tissuepars);

%%% Now simulate
ssfp_b1_delta = zeros([n n 3]);
for ii=1:n
    for jj=1:n
        
        dphi=0;
        
        %%% Symmetric (3-band) case
        ssfp_b1_delta(ii,jj,3) = ssSSFP_ihMT(flipangle,b1sqrd{ii,jj,1},deltaf(jj)*[-1 0 1],TR,tau,dphi,tmp);
        %%% Asymmetric (2-band) case
        ssfp_b1_delta(ii,jj,2) = ssSSFP_ihMT(flipangle,b1sqrd{ii,jj,2},deltaf(jj)*[-1 0 1],TR,tau,dphi,tmp);
        %%% Single band case
        ssfp_b1_delta(ii,jj,1) =ssSSFP_ihMT(flipangle,b1sqrd{ii,jj,1}.*[0 1 0],deltaf(jj)*[-1 0 1],TR,tau,dphi,tmp);
        
        disp([ii jj])
    end
end

%
figfp(1)
imagesc(deltaf*1e-3,b1rms,abs(ssfp_b1_delta(:,:,2)-ssfp_b1_delta(:,:,3))./abs(ssfp_b1_delta(:,:,3)),[0 0.15])
xlabel('\Delta/2\pi, kHz')
ylabel('B_1^{rms}, uT')
colorbar
set(gca,'fontsize',12)
title ('ihMT ratio vs pulse parameters')


%% Comparison for different frequency offsets and flip angles, SSFP only

%%% Set up sequence

%%% generate some pulses
flipangle = deg2rad(linspace(10,80,n));

n=32;
b1rmsfix = 5;
deltaf= linspace(1000,16e3,n);

tau = 2e-3;
TR = 5e-3;
dt = 10e-6;

pulses = {};
b1sqrd = {};
b1pulse= {};

for ii=1:n
    for jj=1:n
        % 3 bands
        [pulses{ii,jj,1},b1sqrd{ii,jj,1},b1pulse{ii,jj,1},TBP] = gen_MB_pulse(flipangle(ii),tau,TR,b1rmsfix,deltaf(jj),'3','alpha',gauss_alpha,'dt',dt);
        % 2 bands
        [pulses{ii,jj,2},b1sqrd{ii,jj,2},b1pulse{ii,jj,2},TBP] = gen_MB_pulse(flipangle(ii),tau,TR,b1rmsfix,deltaf(jj),'2+','alpha',gauss_alpha,'dt',dt);
    end
end

%%% Bieri-Scheffler correction
tmp = Bieri_Scheffler_finite_correction(TBP,tau,TR,tissuepars);

%%% Now simulate
ssfp_fa_delta = zeros([n n 3]);
for ii=1:n
    for jj=1:n
        
        dphi=0;
        
        %%% Symmetric (3-band) case
        ssfp_fa_delta(ii,jj,3) = ssSSFP_ihMT(flipangle(ii),b1sqrd{ii,jj,1},deltaf(jj)*[-1 0 1],TR,tau,dphi,tmp);
        %%% Asymmetric (2-band) case
        ssfp_fa_delta(ii,jj,2) = ssSSFP_ihMT(flipangle(ii),b1sqrd{ii,jj,2},deltaf(jj)*[-1 0 1],TR,tau,dphi,tmp);
        %%% Single band case
        ssfp_fa_delta(ii,jj,1) =ssSSFP_ihMT(flipangle(ii),b1sqrd{ii,jj,1}.*[0 1 0],deltaf(jj)*[-1 0 1],TR,tau,dphi,tmp);
        
        disp([ii jj])
    end
end

%%

In [ ]:
%get nr --from MATLAB
%get nc --from MATLAB
%get Mssfp --from MATLAB
%get flips --from MATLAB
%get Mspgr --from MATLAB


In [ ]:

fig = make_subplots(rows=nr, cols=nc, subplot_titles=['bSSFP', 'sPGR', 'bSSFP', 'sPGR'])

# first plot delta=0, bSSFP
fig.add_trace(go.Scatter(x=np.rad2deg(flips), y=abs(Mssfp[:,0,0]), line_color='black', showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(x=np.rad2deg(flips), y=abs(Mssfp[:,1,0]), fill='tonexty', line = dict(shape = 'linear', color = 'royalblue', width= 4, dash = 'dash'),showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(x=np.rad2deg(flips), y=abs(Mssfp[:,2,0]), line_color='lightgreen', showlegend=False), row=1, col=1)


fig.add_trace(go.Scatter(x=np.rad2deg(flips), y=abs(Mspgr[:,0,0]), line_color='black', showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=np.rad2deg(flips), y=abs(Mspgr[:,1,0]), fill='tonexty', line_color='royalblue', showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=np.rad2deg(flips), y=abs(Mspgr[:,2,0]),line = dict(shape = 'linear', color = 'lightgreen', width= 4, dash = 'dash'), showlegend=False), row=1, col=2)

fig.add_trace(go.Scatter(x=np.rad2deg(flips), y=abs(Mssfp[:,0,1]), line_color='black', showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(x=np.rad2deg(flips), y=abs(Mssfp[:,2,1]), line=dict(color='royalblue', dash='dash', width=4), showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(x=np.rad2deg(flips), y=abs(Mssfp[:,1,1]), fill='tonexty', line_color='lightgreen', showlegend=False), row=2, col=1)


fig.add_trace(go.Scatter(x=np.rad2deg(flips), y=abs(Mspgr[:,0,1]), line_color='black', name=r"S<sub>1B</sub>"), row=2, col=2)
fig.add_trace(go.Scatter(x=np.rad2deg(flips), y=abs(Mspgr[:,2,1]), line=dict(color='royalblue', dash='dash', width=4), name=r"S<sub>3B</sub>"), row=2, col=2)
fig.add_trace(go.Scatter(x=np.rad2deg(flips), y=abs(Mspgr[:,1,1]), fill='tonexty', line_color='lightgreen',name=r"S<sub>2B</sub>"), row=2, col=2)
    

fig.update_layout(width=800, height=600)
fig.show()

In [69]:


fig = make_subplots(rows=1, cols=2, subplot_titles=['bSSFP', 'sPGR'])

# first plot delta=0, bSSFP
fig.add_scatter(x=np.rad2deg(flips), y=abs(Mssfp[:,0,0]), line_color='black', showlegend=False, row=1, col=1, visible=True)
fig.add_scatter(x=np.rad2deg(flips), y=abs(Mssfp[:,1,0]), fill='tonexty', line = dict(shape = 'linear', color = 'royalblue', width= 4, dash = 'dash'),showlegend=False, row=1, col=1, visible=True)
fig.add_scatter(x=np.rad2deg(flips), y=abs(Mssfp[:,2,0]), line_color='lightgreen', showlegend=False, row=1, col=1, visible=True)


fig.add_scatter(x=np.rad2deg(flips), y=abs(Mspgr[:,0,0]), line_color='black', showlegend=False, row=1, col=2, visible=True)
fig.add_scatter(x=np.rad2deg(flips), y=abs(Mspgr[:,1,0]), fill='tonexty', line_color='royalblue', showlegend=False, row=1, col=2, visible=True)
fig.add_scatter(x=np.rad2deg(flips), y=abs(Mspgr[:,2,0]),line = dict(shape = 'linear', color = 'lightgreen', width= 4, dash = 'dash'), showlegend=False, row=1, col=2, visible=True)

fig.add_scatter(x=np.rad2deg(flips), y=abs(Mssfp[:,0,1]), line_color='black', showlegend=False, row=1, col=1, visible=False)
fig.add_scatter(x=np.rad2deg(flips), y=abs(Mssfp[:,2,1]), line=dict(color='royalblue', dash='dash', width=4), showlegend=False, row=1, col=1, visible=False)
fig.add_scatter(x=np.rad2deg(flips), y=abs(Mssfp[:,1,1]), fill='tonexty', line_color='lightgreen', showlegend=False, row=1, col=1, visible=False)

fig.add_scatter(x=np.rad2deg(flips), y=abs(Mspgr[:,0,1]), line_color='black', name=r"S<sub>1B</sub>", row=1, col=2, visible='legendonly')
fig.add_scatter(x=np.rad2deg(flips), y=abs(Mspgr[:,2,1]), line=dict(color='royalblue', dash='dash', width=4), name=r"S<sub>3B</sub>", row=1, col=2, visible='legendonly')
fig.add_scatter(x=np.rad2deg(flips), y=abs(Mspgr[:,1,1]), fill='tonexty', line_color='lightgreen',name=r"S<sub>2B</sub>", row=1, col=2, visible='legendonly')
    
fig['layout']['xaxis']['title'] = f'Flip angle, deg'
fig['layout']['yaxis']['title'] = f'Signal / M \N{SUBSCRIPT ZERO}'
fig['layout']['xaxis2']['title'] = f'Flip angle, deg'
fig['layout']['yaxis2']['title'] = f'Signal / M \N{SUBSCRIPT ZERO}'


steps = []
for i in [0,6]:
    step = dict(
        method = 'restyle',  
        args = ['visible', ['legendonly'] * len(fig.data)],
        label=f'\N{GREEK SMALL LETTER DELTA}={0 if i is 0 else 1}',
    )
    step['args'][1][i] = True
    step['args'][1][i+1] = True
    step['args'][1][i+2] = True
    
    step['args'][1][i+3] = True
    step['args'][1][i+4] = True
    step['args'][1][i+5] = True
    steps.append(step)

sliders = [dict(
    steps = steps,
)]

fig.layout.sliders = sliders
fig.show()
    
# fig.update_layout(width=1200, height=500)
# fig.show()

In [76]:
% Script to test the instantaneous pulse approximation against time
% integration of the BMP equations directly
% Shaihan Malik, King's College London, 2019

tissuepars = init_tissue('ic');


%%% Set up sequence

%%% generate some pulses
flips = deg2rad([1:2:30 35:5:80]);
nf = length(flips);
tau = 2e-3;
TR = 5e-3;
b1_rms = 5;
delta = 7e3;
dt = 10e-6;
gauss_alpha = 3; %<-- width parameter for pulses

pulses = {};
b1sqrd = {};
b1pulse= {};
for ii=1:nf
    % 3 bands
    [pulses{ii,1},b1sqrd{ii,1},b1pulse{ii,1},TBP] = gen_MB_pulse(flips(ii),tau,TR,b1_rms,delta,'3','alpha',gauss_alpha,'dt',dt); 
    % 2 bands
    [pulses{ii,2},b1sqrd{ii,2},b1pulse{ii,2},TBP] = gen_MB_pulse(flips(ii),tau,TR,b1_rms,delta,'2+','alpha',gauss_alpha,'dt',dt);
end


%% bSSFP steady state

%%% Extra pars needed for SSFP
dphi=0;

% loop over the different flips
Mssfp = zeros([nf 3 3]); % Flips, #bands, type of simulation

Delta_Hz = [-delta 0 delta];
for ii=1:nf
    
    %%% ===== Instantaneous Assumption  ======    
    %%% Symmetric (3-band) case
    [Mssfp(ii,3,1)] = ssSSFP_ihMT(flips(ii),b1sqrd{ii,1},Delta_Hz,TR,tau,dphi,tissuepars);
    %%% Asymmetric (2-band) case
    [Mssfp(ii,2,1)] = ssSSFP_ihMT(flips(ii),b1sqrd{ii,2},Delta_Hz,TR,tau,dphi,tissuepars);
    %%% Single band case
    Mssfp(ii,1,1) =ssSSFP_ihMT(flips(ii),b1sqrd{ii,1}.*[0 1 0],Delta_Hz,TR,tau,dphi,tissuepars);
    
    %%% ===== Full integration  ======
    %%% Symmetric (3-band) case
    [Mssfp(ii,3,2)] = ssSSFP_ihMT_integrate(b1pulse{ii,1},dt,Delta_Hz,TR,dphi,tissuepars);
    %%% Asymmetric (2-band) case
    [Mssfp(ii,2,2)] = ssSSFP_ihMT_integrate(b1pulse{ii,2},dt,Delta_Hz,TR,dphi,tissuepars);
    %%% Single band case
    Mssfp(ii,1,2) =ssSSFP_ihMT_integrate(b1pulse{ii,1}.*[0 1 0],dt,Delta_Hz,TR,dphi,tissuepars);

    %%% ===== Bieri Scheffler correction
    
    % compute new R2
    tissuepars_mod = Bieri_Scheffler_finite_correction(TBP,tau,TR,tissuepars);

    %%% Symmetric (3-band) case
    [Mssfp(ii,3,3)] = ssSSFP_ihMT(flips(ii),b1sqrd{ii,1},Delta_Hz,TR,tau,dphi,tissuepars_mod);
    %%% Asymmetric (2-band) case
    [Mssfp(ii,2,3)] = ssSSFP_ihMT(flips(ii),b1sqrd{ii,2},Delta_Hz,TR,tau,dphi,tissuepars_mod);
    %%% Single band case
    Mssfp(ii,1,3) =ssSSFP_ihMT(flips(ii),b1sqrd{ii,1}.*[0 1 0],Delta_Hz,TR,tau,dphi,tissuepars_mod);
    
end

%% Repeat above, but for SPGR, miss out correction on here

% loop over the different flips
Mspgr = zeros([nf 3 2]); % Flips, #bands, type of simulation

Delta_Hz = [-delta 0 delta];
for ii=1:nf
    
    %%% ===== Instantaneous Assumption  ======    
    %%% Symmetric (3-band) case
    [Mspgr(ii,3,1)] = ssSPGR_ihMT(flips(ii),b1sqrd{ii,1},Delta_Hz,TR,tau,tissuepars);
    %%% Asymmetric (2-band) case
    [Mspgr(ii,2,1)] = ssSPGR_ihMT(flips(ii),b1sqrd{ii,2},Delta_Hz,TR,tau,tissuepars);
    %%% Single band case
    Mspgr(ii,1,1) =ssSPGR_ihMT(flips(ii),b1sqrd{ii,1}.*[0 1 0],Delta_Hz,TR,tau,tissuepars);
    
    %%% ===== Full integration  ======
    %%% Symmetric (3-band) case
    [Mspgr(ii,3,2)] = ssSPGR_ihMT_integrate(b1pulse{ii,1},dt,Delta_Hz,TR,tissuepars);
    %%% Asymmetric (2-band) case
    [Mspgr(ii,2,2)] = ssSPGR_ihMT_integrate(b1pulse{ii,2},dt,Delta_Hz,TR,tissuepars);
    %%% Single band case
    Mspgr(ii,1,2) =ssSPGR_ihMT_integrate(b1pulse{ii,1}.*[0 1 0],dt,Delta_Hz,TR,tissuepars);

    
end




%% Now consider a fixed flip angle, look at different pulse duration and dipolar relaxation time

%%% generate some pulses
flipangle = deg2rad(60);

% range of tau and dipolar T1
n=16;
tau = linspace(0.4e-3,2.5e-3,n);
T1D = logspace(-4,-2,n);

TR = 5e-3;
b1_rms = 5;
delta = 7e3;
dt = 10e-6;

pulses = {};
b1sqrd = {};
b1pulse= {};
for ii=1:n
    % 3 bands
    [pulses{ii,1},b1sqrd{ii,1},b1pulse{ii,1}] = gen_MB_pulse(flipangle,tau(ii),TR,b1_rms,delta,'3','alpha',gauss_alpha,'dt',dt); 
    % 2 bands
    [pulses{ii,2},b1sqrd{ii,2},b1pulse{ii,2}] = gen_MB_pulse(flipangle,tau(ii),TR,b1_rms,delta,'2+','alpha',gauss_alpha,'dt',dt);
end


%%% Now compute a grid of points
Mssfp_all = zeros([n n 3 3]); % T1D, tau, method, number of bands

for ii=1:n

    % generate new tissue parameter set
    tmp = init_tissue('ic');
    tmp.semi.R1D = 1/T1D(ii);

    % Now loop over pulses (i.e. different tau)
    for jj=1:n
        
        %==== Standard Steady state method ====
        %%% Symmetric (3-band) case
        Mssfp_all(ii,jj,1,1) = ssSSFP_ihMT(flipangle,b1sqrd{jj,1},Delta_Hz,TR,tau(jj),dphi,tmp);
        %%% Asymmetric (2-band) case
        Mssfp_all(ii,jj,1,2) = ssSSFP_ihMT(flipangle,b1sqrd{jj,2},Delta_Hz,TR,tau(jj),dphi,tmp);
        %%% Single band case
        Mssfp_all(ii,jj,1,3) =ssSSFP_ihMT(flipangle,b1sqrd{jj,1}.*[0 1 0],Delta_Hz,TR,tau(jj),dphi,tmp);
        
        %==== Full integration ====
        %%% Symmetric (3-band) case
        Mssfp_all(ii,jj,2,1) = ssSSFP_ihMT_integrate(b1pulse{jj,1},dt,Delta_Hz,TR,dphi,tmp);
        %%% Asymmetric (2-band) case
        Mssfp_all(ii,jj,2,2) = ssSSFP_ihMT_integrate(b1pulse{jj,2},dt,Delta_Hz,TR,dphi,tmp);
        %%% Single band case
        Mssfp_all(ii,jj,2,3) =ssSSFP_ihMT_integrate(b1pulse{jj,1}.*[0 1 0],dt,Delta_Hz,TR,dphi,tmp);
        
        %=== Bieri Scheffler Correction
        tmpC = Bieri_Scheffler_finite_correction(TBP,tau(jj),TR,tmp);

        %%% Symmetric (3-band) case
        Mssfp_all(ii,jj,3,1) = ssSSFP_ihMT(flipangle,b1sqrd{jj,1},Delta_Hz,TR,tau(jj),dphi,tmpC);
        %%% Asymmetric (2-band) case
        Mssfp_all(ii,jj,3,2) = ssSSFP_ihMT(flipangle,b1sqrd{jj,2},Delta_Hz,TR,tau(jj),dphi,tmpC);
        %%% Single band case
        Mssfp_all(ii,jj,3,3) =ssSSFP_ihMT(flipangle,b1sqrd{jj,1}.*[0 1 0],Delta_Hz,TR,tau(jj),dphi,tmpC);
        
       disp([ii jj]) 
    end
end


%% Now do the SPGR version - for Ernst angle (7.1)
flipangle = acos(exp(-TR*tissuepars.free.R1));
fprintf(1,'flip is %1.1f\n',rad2deg(flipangle));
pulses = {};
b1sqrd = {};
b1pulse= {};
for ii=1:n
    % 3 bands
    [pulses{ii,1},b1sqrd{ii,1},b1pulse{ii,1}] = gen_MB_pulse(flipangle,tau(ii),TR,b1_rms,delta,'3','alpha',gauss_alpha,'dt',dt); 
    % 2 bands
    [pulses{ii,2},b1sqrd{ii,2},b1pulse{ii,2}] = gen_MB_pulse(flipangle,tau(ii),TR,b1_rms,delta,'2+','alpha',gauss_alpha,'dt',dt);
end


Mspgr_all = zeros([n n 2 3]); % T1D, tau, method, number of bands

for ii=1:n

    % generate new tissue parameter set
    tmp = init_tissue('ic');
    tmp.semi.R1D = 1/T1D(ii);

    % Now loop over pulses (i.e. different tau)
    for jj=1:n
        
        %==== Standard Steady state method ====
        %%% Symmetric (3-band) case
        Mspgr_all(ii,jj,1,1) = ssSPGR_ihMT(flipangle,b1sqrd{jj,1},Delta_Hz,TR,tau(jj),tmp);
        %%% Asymmetric (2-band) case
        Mspgr_all(ii,jj,1,2) = ssSPGR_ihMT(flipangle,b1sqrd{jj,2},Delta_Hz,TR,tau(jj),tmp);
        %%% Single band case
        Mspgr_all(ii,jj,1,3) =ssSPGR_ihMT(flipangle,b1sqrd{jj,1}.*[0 1 0],Delta_Hz,TR,tau(jj),tmp);
        
        %==== Full integration ====
        %%% Symmetric (3-band) case
        Mspgr_all(ii,jj,2,1) = ssSPGR_ihMT_integrate(b1pulse{jj,1},dt,Delta_Hz,TR,tmp);
        %%% Asymmetric (2-band) case
        Mspgr_all(ii,jj,2,2) = ssSPGR_ihMT_integrate(b1pulse{jj,2},dt,Delta_Hz,TR,tmp);
        %%% Single band case
        Mspgr_all(ii,jj,2,3) =ssSPGR_ihMT_integrate(b1pulse{jj,1}.*[0 1 0],dt,Delta_Hz,TR,tmp);
        
        
        
       disp([ii jj]) 
    end
end





%% Make a big figure

figfp(100)

nr = 3;
nc = 4;

%%% Plots
subplot(nr,nc,1)
plot(rad2deg(flips),squeeze(abs(Mspgr(:,2,:))),'linewidth',1.5)
grid on
pl=get(gca,'children');
pl(1).Color = [0 0 0];
legend('Instantaneous, Eq.[5]','Full integration')
ylim([0 0.04])
ylabel('Signal/M_0')
xlabel('flip, deg')
title('SPGR (2B)')

subplot(nr,nc,nc+1)
plot(rad2deg(flips),squeeze(abs(Mssfp(:,2,:))),'linewidth',1.5)
grid on
pl=get(gca,'children');
pl(1).LineStyle='-.';
pl(1).Color = [1 0.2 0.2];
pl(2).Color = [0 0 0];
ll=legend('Instantaneous, Eq.[6]','Full integration','Bieri-Scheffler correction');
ll.Position = [0.1032 0.1338 0.1700 0.0967];
ylim([0 0.12])
ylabel('Signal/M_0')
xlabel('flip, deg')
title('bSSFP (2B)')

%%% Now SPGR, no correction
titls = {'1B','2B','3B'};
for ii=1:3    
    subplot(nr,nc,ii+1)
    [cc,hh]=contourf(1e3*tau,T1D,100*abs(abs(Mspgr_all(:,:,2,ii))-abs(Mspgr_all(:,:,1,ii)))./abs(Mspgr_all(:,:,2,ii)),0:2:10);
    hh.LineWidth = 1.5;
    clabel(cc,hh,'Color',[1 1 1]);
    title(titls{ii},'fontsize',18)
    set(gca,'Yscale','log')
    grid on
    caxis([0 10])
    hh.LineColor = [1 1 1];
    
    %xlabel('Pulse duration \tau (ms)')
    xlabel('\tau, ms')
    ylabel('T_{1D}^s (s)')
end

%%% SSFP, with and without correction
for ii=1:3    
    subplot(nr,nc,ii+1+nc)
    [cc,hh]=contourf(1e3*tau,T1D,100*abs(abs(Mssfp_all(:,:,2,ii))-abs(Mssfp_all(:,:,1,ii)))./abs(Mssfp_all(:,:,2,ii)),0:2:10);
    hh.LineWidth = 1.5;
    clabel(cc,hh,'Color',[1 1 1]);
    
    %title(sprintf('%d band, error',ii))
    set(gca,'Yscale','log')
    grid on
    caxis([0 10])
    hh.LineColor = [1 1 1];
    
   % xlabel('Pulse duration \tau (ms)')
    ylabel('T_{1D}^s (s)')
    
    % Bieri Scheffler correction
    subplot(nr,nc,ii+1+2*nc)
    [cc,hh]=contourf(1e3*tau,T1D,100*abs(abs(Mssfp_all(:,:,2,ii))-abs(Mssfp_all(:,:,3,ii)))./abs(Mssfp_all(:,:,2,ii)),0:2:10);
    %title(sprintf('%d band, error',ii))
    hh.LineWidth = 1.5;
    clabel(cc,hh,'Color',[1 1 1]);
    set(gca,'Yscale','log')
    grid on
    caxis([0 10])
    hh.LineColor = [1 1 1];
    
    %xlabel('Pulse duration \tau (ms)')
    xlabel('\tau, ms')
    ylabel('T_{1D}^s (s)')
    
    
end
% colormap hot

setpospap([100 100 1100 635])

gg = get(gcf,'children');

% Reposition everything
gg(13).Position = [0.06    0.59    0.22    0.35];
gg(11).Position = [0.06    0.11    0.22    0.35];

% plots
ix = [9 8 7];
ix2 = [6 4 2];
ix3 = [5 3 1];

for ii=1:3
    gg(ix(ii)).Position = [0.40+(ii-1)*0.185 0.68 0.13 0.2];
    gg(ix2(ii)).Position = [0.40+(ii-1)*0.185 0.35 0.13 0.2];
    gg(ix3(ii)).Position = [0.40+(ii-1)*0.185 0.08 0.13 0.2];
end

%%% text labels
axes(gg(13))
text(140,0.055,'Percentage error from instantaneous approximation','fontsize',13,'fontweight','bold');

text(90,0.015,'SPGR','rotation',90,'fontsize',14,'fontweight','bold')
text(97,0.0085,'instantaneous','rotation',90,'fontsize',13,'fontweight','bold')


text(90,-0.04,'bSSFP','rotation',90,'fontsize',14,'fontweight','bold')
text(97,-0.028,'instantaneous','rotation',90,'fontsize',13,'fontweight','bold')
text(97,-0.055,'corrected','rotation',90,'fontsize',13,'fontweight','bold')

text(307,-0.015,'% deviation','rotation',0,'fontsize',13,'fontweight','bold')


%%% Color bar
axes(gg(1))
cc = colorbar;
cc.Position = [0.9338 0.4850 0.0152 0.2500];

% print -dpng -r300 figs/instantaneous_approx_fig.png


%% Additional material - make a comparison with MAMT style integration where we
% integrate over multiple TRs to reach steady state

% arbitrarily choose pulse
II=13;

% Now let's work out speed up factor
tt1 = [];
for ii=1:100
    tic;[aa bb] = ssSSFP_ihMT_integrate(b1pulse{II,2},dt,Delta_Hz,TR,dphi,tissuepars);tt1(ii)=toc;
end

NTR = [100 200 300 400 500];
n = 10; %number of repeats
mamt = zeros([n 5]);
tt2 = zeros([n 5]);
for ii=1:n
    for jj=1:5
        tic;[cc dd mt] = ssSSFP_ihMT_integrate_MAMT(b1pulse{II,2},dt,Delta_Hz,TR,dphi,NTR(jj),tissuepars);tt2(ii,jj)=toc;
        mamt(ii,jj)=abs([1 1i 0 0 0 0]*mt(:,end));
        disp([ii jj])
    end
end
save bin/mamt_comparison mamt NTR tt1 tt2 aa

%
figfp(1);


subplot(1,2,1)
plot(NTR,100*(mean(mamt,1)-abs(aa))/abs(aa),'linewidth',2)
grid on
xlabel('Number of TR periods')
ylabel('Percentage deviation')
title('Error from not reaching steady-state')

subplot(1,2,2)
errorbar(NTR,mean(tt2,1)/mean(tt1),std(tt2,[],1)/mean(tt1),'linewidth',2)
grid on
xlabel('Number of TR periods')
ylabel('Relative speed increase')
title('Relative speed increase of eigenvalue method')
xlim([0 550])
ylim([0 550])


% print -dpng -r300 figs/mamt_comparison_fig.png


In [87]:
%get nr --from MATLAB
%get nc --from MATLAB
%get flips --from MATLAB
%get Mspgr --from MATLAB
%get Mssfp --from MATLAB

In [111]:
# plt.plot(np.rad2deg(flips), np.squeeze(np.abs(Mspgr[:,2,:])))


fig = make_subplots(rows=nr, cols=nc)

# first plot delta=0, bSSFP
fig.add_trace(go.Scatter(x=np.rad2deg(flips), y=np.squeeze(np.abs(Mspgr[:,2,0])), line_color='royalblue', name='Instantenuous, Eq.[5]'), row=1, col=1)
fig.add_trace(go.Scatter(x=np.rad2deg(flips), y=np.squeeze(np.abs(Mspgr[:,2,1])), line_color='black', name='Full Integration'), row=1, col=1)

fig.add_trace(go.Scatter(x=np.rad2deg(flips), y=np.squeeze(np.abs(Mssfp[:,2,0])), line_color='blue', name='Instantenuous, Eq.[6]'), row=2, col=1)
fig.add_trace(go.Scatter(x=np.rad2deg(flips), y=np.squeeze(np.abs(Mssfp[:,2,1])), line_color='black', name='Full Integration'), row=2, col=1)
fig.add_trace(go.Scatter(x=np.rad2deg(flips), y=np.squeeze(np.abs(Mssfp[:,2,2])), line_color='red', name='Bieri-Scheffler correction', line = dict(shape = 'linear', color = 'red', width= 4, dash = 'dot')), row=2, col=1)

fig.update_layout(width=1600, height=800)
fig.show()

In [96]:

%%% Now SPGR, no correction
titls = {'1B','2B','3B'};
for ii=1:3    
    subplot(nr,nc,ii+1)
    [cc,hh]=contourf(1e3*tau,T1D,100*abs(abs(Mspgr_all(:,:,2,ii))-abs(Mspgr_all(:,:,1,ii)))./abs(Mspgr_all(:,:,2,ii)),0:2:10);
    hh.LineWidth = 1.5;
    clabel(cc,hh,'Color',[1 1 1]);
    title(titls{ii},'fontsize',18)
    set(gca,'Yscale','log')
    grid on
    caxis([0 10])
    hh.LineColor = [1 1 1];
    
    %xlabel('Pulse duration \tau (ms)')
    xlabel('\tau, ms')
    ylabel('T_{1D}^s (s)')
end

%%% SSFP, with and without correction
for ii=1:3    
    subplot(nr,nc,ii+1+nc)
    [cc,hh]=contourf(1e3*tau,T1D,100*abs(abs(Mssfp_all(:,:,2,ii))-abs(Mssfp_all(:,:,1,ii)))./abs(Mssfp_all(:,:,2,ii)),0:2:10);
    hh.LineWidth = 1.5;
    clabel(cc,hh,'Color',[1 1 1]);
    
    %title(sprintf('%d band, error',ii))
    set(gca,'Yscale','log')
    grid on
    caxis([0 10])
    hh.LineColor = [1 1 1];
    
   % xlabel('Pulse duration \tau (ms)')
    ylabel('T_{1D}^s (s)')
    
    % Bieri Scheffler correction
    subplot(nr,nc,ii+1+2*nc)
    [cc,hh]=contourf(1e3*tau,T1D,100*abs(abs(Mssfp_all(:,:,2,ii))-abs(Mssfp_all(:,:,3,ii)))./abs(Mssfp_all(:,:,2,ii)),0:2:10);
    %title(sprintf('%d band, error',ii))
    hh.LineWidth = 1.5;
    clabel(cc,hh,'Color',[1 1 1]);
    set(gca,'Yscale','log')
    grid on
    caxis([0 10])
    hh.LineColor = [1 1 1];
    
    %xlabel('Pulse duration \tau (ms)')
    xlabel('\tau, ms')
    ylabel('T_{1D}^s (s)')
    
    
end